In [ ]:
import numpy as np
import scipy.io
import mne
import ipywidgets
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse
import matplotlib.pyplot as plt


# 1. 加载MATLAB数据
mat_data = scipy.io.loadmat('name.mat')
eeg_data = mat_data['data']  # 获取32×1000×300的矩阵
# 在加载数据后添加转置操作
eeg_data = np.transpose(eeg_data, (2, 0, 1))  # 从(32,1000,300)变为(300,32,1000)

# 打印数据形状以确认
print(f"数据形状: {eeg_data.shape} (通道×时间点×试次)")

# 2. 创建MNE数据结构
sfreq = 500  # 采样率(Hz)，根据实际设置调整
tmin = 0   # 刺激前起点(秒)，根据实验设置调整

# EEGLAB标准1005系统的32通道名称
ch_names = [
    'Fp1', 'Fp2', 
    'AF3', 'AF4', 
    'F7', 'F3', 'Fz', 'F4', 'F8',
    'FC5', 'FC1', 'FC2', 'FC6',
    'T7', 'C3', 'Cz', 'C4', 'T8',
    'CP5', 'CP1', 'CP2', 'CP6',
    'P7', 'P3', 'Pz', 'P4', 'P8',
    'PO3', 'PO4', 
    'O1', 'Oz', 'O2'
]
    
# 创建标准电极位置 - 使用1005系统蒙太奇

montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')

# 检查montage中的通道是否与info匹配
montage_ch_names = set(montage.ch_names)
info_ch_names = set(info.ch_names)
info.set_montage(montage)

# 3. 创建Epochs对象
n_epochs = eeg_data.shape[0]
events = np.zeros((n_epochs, 3), dtype=int)
events[:, 0] = np.arange(n_epochs) * (eeg_data.shape[1] + 100)  # 确保事件间隔足够
events[:, 2] = 1  # 所有事件类型设为1

# 创建EpochsArray
epochs = mne.EpochsArray(
    data=eeg_data,
    info=info,
    events=events,
    tmin=tmin,
    event_id={'condition': 1},
    baseline=(-0.2, 0) if tmin <= -0.2 else None,
    proj=False  # 暂时不使用投影
)
# 设置平均参考投影器
epochs.set_eeg_reference(ref_channels='average', projection=True)

# 激活并应用投影器
epochs.apply_proj()

# 验证投影器已生效
print("\n激活的投影器:", [p for p in epochs.info["projs"] if p["active"]])

# 打印最终通道信息
print("\n最终通道信息:")
print(f"数据中的通道数: {epochs.info['nchan']}")
print(f"通道名称: {epochs.info['ch_names']}")

In [ ]:
import numpy as np
import scipy.io
import mne
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse
import matplotlib.pyplot as plt

# 1. 加载MATLAB数据
mat_data = scipy.io.loadmat('name.mat')
eeg_data = mat_data['data']  # 获取32×1000×300的矩阵
eeg_data = np.transpose(eeg_data, (2, 0, 1))  # 从(32,1000,300)变为(300,32,1000)

# 2. 创建MNE数据结构
sfreq = 500  # 采样率(Hz)
tmin = 0     # 刺激前起点(秒)

# 标准64通道名称 (国际10-20系统)
ch_names_64 = [
    'Fp1', 'Fpz', 'Fp2', 
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2', 'Iz'
]

# 原始32通道名称
ch_names_32 = [
    'Fp1', 'Fp2', 
    'AF3', 'AF4', 
    'F7', 'F3', 'Fz', 'F4', 'F8',
    'FC5', 'FC1', 'FC2', 'FC6',
    'T7', 'C3', 'Cz', 'C4', 'T8',
    'CP5', 'CP1', 'CP2', 'CP6',
    'P7', 'P3', 'Pz', 'P4', 'P8',
    'PO3', 'PO4', 
    'O1', 'Oz', 'O2'
]

# 创建64通道数据数组
n_epochs, n_channels_32, n_times = eeg_data.shape
eeg_data_64 = np.zeros((n_epochs, len(ch_names_64), n_times))

# 将32通道数据映射到64通道中的对应位置
for i, ch in enumerate(ch_names_32):
    if ch in ch_names_64:
        idx_64 = ch_names_64.index(ch)
        eeg_data_64[:, idx_64, :] = eeg_data[:, i, :]

# 创建标准电极位置 - 使用1005系统蒙太奇
montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names=ch_names_64, sfreq=sfreq, ch_types='eeg')
info.set_montage(montage)

# 3. 创建Epochs对象
n_epochs = eeg_data_64.shape[0]
events = np.zeros((n_epochs, 3), dtype=int)
events[:, 0] = np.arange(n_epochs) * (eeg_data_64.shape[2] + 100)  # 确保事件间隔足够
events[:, 2] = 1  # 所有事件类型设为1

# 创建EpochsArray
epochs = mne.EpochsArray(
    data=eeg_data_64,
    info=info,
    events=events,
    tmin=tmin,
    event_id={'condition': 1},
    baseline=(-0.2, 0) if tmin <= -0.2 else None,
    proj=False
)

# 设置平均参考投影器
epochs.set_eeg_reference(ref_channels='average', projection=True)
epochs.apply_proj()

# 打印通道信息
print("\n最终通道信息:")
print(f"数据通道数: {epochs.info['nchan']}")
print(f"通道名称: {epochs.info['ch_names']}")

# 4. 获取模板大脑文件
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent
subject = 'fsaverage'

# 确保已获取fsaverage路径
subjects_dir = fetch_fsaverage(verbose=True).parent
mne.datasets.fetch_fsaverage(subjects_dir)

# 5. 创建正向模型 (使用模板)
src = mne.setup_source_space(
    subject, 
    spacing='ico4',
    subjects_dir=subjects_dir,
    add_dist=False
)

# 三层电导率模型
conductivity = (0.3, 0.006, 0.3)  # 大脑/颅骨/头皮
model = mne.make_bem_model(
    subject=subject,
    ico=4,
    conductivity=conductivity,
    subjects_dir=subjects_dir
)
bem = mne.make_bem_solution(model)

# 创建坐标变换
trans = mne.read_trans(f'{subjects_dir}/fsaverage/bem/fsaverage-trans.fif')

# 创建正向解
fwd = mne.make_forward_solution(
    epochs.info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=1
)

# 6. 计算噪声协方差
noise_cov = mne.compute_covariance(
    epochs,
    tmin=tmin,      # 使用整个基线期
    tmax=0.0,
    method='shrunk'
)

# 7. 创建逆算子
inv = make_inverse_operator(epochs.info, fwd, noise_cov)

# 8. 应用dSPM源定位
# 选择第一个试次作为示例
epoch = epochs[0]
stc = apply_inverse(epoch, inv, method='dSPM')

# 9. 可视化源定位结果
# 在fsaverage大脑上绘制源激活
brain = stc.plot(subject='fsaverage', subjects_dir=subjects_dir, 
                hemi='both', views='lateral', time_viewer=True)
brain.add_annotation('aparc', borders=True)

plt.show()

In [ ]:
epochs.plot_sensors(show_names=True, kind='topomap', 
                   title='EEG电极定位 (standard_1005)',
                   ch_groups='position',
                   
                   show=True)
plt.show()

In [ ]:


# 3. 获取模板大脑文件
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent
subject = 'fsaverage'


# 确保已获取fsaverage路径（关键步骤）
subjects_dir = fetch_fsaverage(verbose=True).parent  # 获取自动下载的模板路径
mne.datasets.fetch_fsaverage(subjects_dir)  # 确认模板存在


In [ ]:
# 4. 创建正向模型 (使用模板)
src = mne.setup_source_space(
    subject, 
    spacing='ico4',
    subjects_dir=subjects_dir,
    add_dist=False
)

#conductivity = (0.3,)  # 单层模型
conductivity = (0.3, 0.006, 0.3)
model = mne.make_bem_model(
    subject=subject,
    ico=4,
    conductivity=conductivity,
    subjects_dir=subjects_dir
)
bem = mne.make_bem_solution(model)

# 创建坐标变换 (模板到标准空间)
#trans = mne.read_trans(f'{fs_dir}/fsaverage-trans.fif')
trans = mne.read_trans(f'C:/Users/39397/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-trans.fif')


fwd = mne.make_forward_solution(
    info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=1
)

# 5. 计算噪声协方差
noise_cov = mne.compute_covariance(
    epochs,
    tmin=tmin,      # 使用整个基线期
    tmax=0.0,
    method='shrunk'
)

# 6. 创建逆算子
inv = make_inverse_operator(
    info,
    fwd,
    noise_cov,
    loose=0.2,
    depth=0.8
)

# 7. 应用dSPM到平均诱发电位
evoked = epochs.average()
stc = apply_inverse(
    evoked,
    inv,
    lambda2=1./9.,
    method='dSPM'
)

fig, ax = plt.subplots()
ax.plot(1e3 * stc.times, stc.data[::100, :].T)
#ax.set(xlabel="time (ms)", ylabel=f"{method} value")

In [ ]:
# 8. 可视化结果
# 激活大脑图
brain = stc.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    hemi='both',
    initial_time=0.1,  # 显示100ms时刻
    clim=dict(kind='percent', lims=[90, 95, 99]),
    views=['lateral', 'medial'],
    size=(800, 600)
)

# 保存结果
stc.save('dSPM_source_estimate', overwrite=True)
brain.save_image('source_activation.png', overwrite=True)

# 时间序列图
plt.figure(figsize=(10, 6))
stc.plot()
plt.title('dSPM Source Time Course')
plt.savefig('source_time_course.png')